In [35]:
import requests 
import bs4
import pandas as pd
import numpy as np

In [36]:
EDIBLE_FRUITS_URL = 'https://en.wikipedia.org/wiki/List_of_culinary_fruits'

In [37]:
response = requests.get(EDIBLE_FRUITS_URL)
if response.status_code == 200:
    print ("Get Successfull")
else:
    print ("Get Failed")

Get Successfull


In [38]:
resp_soap = bs4.BeautifulSoup(response.text)

In [39]:
resp_soap.title

<title>List of culinary fruits - Wikipedia</title>

In [40]:
resp_soap.table

<table class="wikitable sortable">
<tbody><tr>
<th><a href="/wiki/Common_name" title="Common name">Common name</a>
</th>
<th><a href="/wiki/Binomial_nomenclature" title="Binomial nomenclature">Species name</a>
</th>
<th><a href="/wiki/Cultivar" title="Cultivar">Cultivar list</a>
</th></tr>
<tr>
<td><a href="/wiki/Apple" title="Apple">Apple</a>
</td>
<td><i>Malus pumila</i>
</td>
<td><a href="/wiki/List_of_apple_cultivars" title="List of apple cultivars">Apple cultivars</a>
</td></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/Chinese_quince" title="Chinese quince">Chinese quince</a>
</td>
<td><i>Pseudocydonia sinensis</i>
</td>
<td>
</td></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/Chokeberry" title="Chokeberry">Chokeberry</a>
</td>
<td><i>Aronia melanocarpa</i>
</td>
<td>
</td></tr>
<tr>
<td><a class="mw-redirect" href="/wiki/Cocky_apple" title="Cocky apple">Cocky apple</a>
</td>
<td><i>Planchonia careya</i>
</td>
<td>
</td></tr>
<tr>
<td><a href="/wiki/Crataegus_aestivalis" tit

In [41]:
def add_to_dictionary(table_dict):
    
    if (row_list[0]) in table_dict:
        table_dict[row_list[0]][1] = pos_dict[row_list[0]][1] + ", " + row_list[2]
    else:
        table_dict[row_list[0]] = [row_list[1],row_list[2]]

In [53]:
def process_row(row):
    
    row_values = []    
    for column in row:
        
        if isinstance(column,bs4.element.Tag): # Avoid processing NavigableString - Mostly newline char      
            row_values.append(column.get_text(strip=True))
            
    return row_values

In [54]:
def process_table(soap_msg_table):
    
    row_list = []
    for row in soap_msg_table:
        
        if isinstance(row,bs4.element.Tag): # Avoid processing NavigableString - Mostly newline char
            row_list.append(process_row(row))
            
    print (row_list)
    return row_list

In [56]:
common_spices = pd.DataFrame(process_table(resp_soap.table.tbody))

[['Common name', 'Species name', 'Cultivar list'], ['Apple', 'Malus pumila', 'Apple cultivars'], ['Chinese quince', 'Pseudocydonia sinensis', ''], ['Chokeberry', 'Aronia melanocarpa', ''], ['Cocky apple', 'Planchonia careya', ''], ['Eastern mayhaw', 'Crataegus aestivalis', ''], ['Hawthorn', 'Crataegus rhipidophylla', ''], ['Jagua', 'Genipa americana', ''], ['Loquat', 'Eriobotrya japonica', ''], ['Lovi-lovi', 'Flacourtia inermis', ''], ['Medlar', 'Mespilus germanica', ''], ["Niedzwetzky's apple", 'Malus niedzwetzkyana', ''], ['Pear', 'Pyrus communis', 'Pear cultivars'], ['Quince', 'Cydonia oblonga', ''], ['Ramontchi', 'Flacourtia indica', ''], ['Rose hip', 'Rosa rugosa', ''], ['Rowan', 'Sorbus aucuparia', ''], ['Sapodilla', 'Manilkara zapota', ''], ['Serviceberry', 'Amelanchier alnifolia', ''], ['Shipova', '× Sorbopyrus irregularis', ''], ['Sorb', 'Sorbus domestica', ''], ['Southern crabapple', 'Malus angustifolia', ''], ['Toyon', 'Heteromeles arbutifolia', '']]


In [57]:
common_spices

,0,1,2
0,Common name,Species name,Cultivar list
1,Apple,Malus pumila,Apple cultivars
2,Chinese quince,Pseudocydonia sinensis,
3,Chokeberry,Aronia melanocarpa,
4,Cocky apple,Planchonia careya,
5,Eastern mayhaw,Crataegus aestivalis,
6,Hawthorn,Crataegus rhipidophylla,
7,Jagua,Genipa americana,
8,Loquat,Eriobotrya japonica,
9,Lovi-lovi,Flacourtia inermis,
